# HW 1

>Or Livne ID 203972922

>Roy Rubin ID 201312907

## Task 1: CSV and SQL
1. Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv”
2. Write Python code to load “mydata.csv” into “mydata” table.
3. Write 2 different SQL statements with different condiJons to retrieve different rows. Explain which parts of the statement are predicate and which parts are projecJon.

### Installations

In [1]:
import sys
print(f'Install pandas\n')
!{sys.executable} -m pip install pandas
print(f'\nInstall pyarrow\n')
!{sys.executable} -m pip install pyarrow
print(f'\nInstall dask\n')
!{sys.executable} -m pip install dask
print(f'\nFinished Installations')

Install pandas


Install pyarrow


Install dask


Finished Installations


### Imports

In [2]:
import sqlite3
import pandas as pd
import numpy as np 
import random
import os 
import gc
import pyarrow.parquet as pq
import pyarrow as pa
import dask
from dask.dataframe import to_parquet
import csv
from dask.dataframe import from_pandas

### Set global definitions

In [3]:
def init_data_set_configuration():
    # Set the value options
    global max_rows, csv_columns, db_file_name, csv_file_name, columns_type
    global parquet_file_name_using_dask,parquet_file_name_using_pyarrow 
    global parquet_file_name_using_pandas
    max_rows = 1000000   # NOTE: final size should be 1000000
    csv_columns = ['id', 'fruit', 'price', 'color']
    columns_type = ['integer', 'text', 'integer', 'text']

    db_file_name = 'mydb.db'
    csv_file_name = 'mydata.csv'
    parquet_file_name_using_dask = 'mydatadask.parquet'
    parquet_file_name_using_pyarrow = 'mydatapyarrow.parquet'
    parquet_file_name_using_pandas = 'mydatapandas.parquet'

    return

In [4]:
# init_data_set_configuration settings
init_data_set_configuration()

## Preparations for Task1.1
create the csv file

In [5]:
"""
Create local CSV file “mydata.csv” with 1000000 
rows with columns (id, fruit, price, color). 
fruit has one of the values ['Orange', 'Grape', 'Apple', 'Banana','Pineapple', 'Avocado'] 
colors are ['Red', 'Green', 'Yellow', 'Blue']. 
Price should be random integer between 10 and 100. 
Filed id should be an index number starting from 1.
"""
class single_fruit():
    fruit_name = ''
    fruit_price = ''
    fruit_color = ''
    fruit_id = ''
    
    # Set the value options
    fruit_options_list = ['Orange', 'Grape', 'Apple', 'Banana','Pineapple', 'Avocado']
    colors_options_list = ['Red', 'Green', 'Yellow', 'Blue']
    price_options_range = [10,100]
    
    def __init__(self, id):
        self.fruit_name = random.choice(self.fruit_options_list)	
        self.fruit_price = np.random.randint(self.price_options_range[0], self.price_options_range[1])
        self.fruit_color = random.choice(self.colors_options_list)
        self.fruit_id = id

def create_csvdatabase_file(max_rows):
    # this loop generate single fruit
    create_n_list_in_advance = max_rows*[None]
    for i_row_index in range(max_rows):
        i_friut = single_fruit(i_row_index)
        create_n_list_in_advance[i_row_index] = [i_friut.fruit_id, i_friut.fruit_name, 
                                                 i_friut.fruit_price, i_friut.fruit_color]
    
    mydata_df = pd.DataFrame(create_n_list_in_advance, columns = csv_columns )
    mydata_df.to_csv(csv_file_name)
    return mydata_df

In [6]:
# create csv file as data frame 
mydata_df = create_csvdatabase_file(max_rows)

## Task1.1: 
Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv”

In [7]:
def create_db_and_table(db_file_name):
    is_exist = os.path.exists(db_file_name)
    if  is_exist:
        os.remove(db_file_name)
    con = sqlite3.connect(db_file_name)  # This row safely connects to DB and creates the DB if does not exist
        
    cur = con.cursor()
    
    # Create table
    columns_type_list = list(map(lambda x,y: x+' ' + y, csv_columns, columns_type))
    columns_type_list_string = "("+", ".join(map(str, columns_type_list))+")"

    cur.execute(''' 
                CREATE TABLE fruits
                ''' + columns_type_list_string + \
               '''''')
    
    con.commit()
    con.close()
    return con, cur

In [8]:
%%time
# create a data base and table
con, cur = create_db_and_table(db_file_name)

CPU times: user 1.7 ms, sys: 3.41 ms, total: 5.11 ms
Wall time: 4.98 ms


## Task1.2: 
Write Python code to load “mydata.csv” into “mydata” table.

In [9]:
def fill_db_data_base_using_csv_data_base_with_same_keys(mydata_df):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()

    for i_csv_row in range(mydata_df.shape[0]):
        # Take row from csv file
        i_row = mydata_df.iloc[i_csv_row]
        i_row_as_list = i_row.to_list()
        
        i_row_as_list_string = "('"+"','".join(map(str, i_row_as_list))+"')"
        #cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT','100')")

        #Insert a csv row of data base
        cur.execute("INSERT INTO fruits VALUES "  \
                    +i_row_as_list_string + \
                    "")
    con.commit()
    con.close()
    return

In [10]:
%%time
# copy csv data into db file
fill_db_data_base_using_csv_data_base_with_same_keys(mydata_df)

CPU times: user 2min 2s, sys: 809 ms, total: 2min 3s
Wall time: 2min 5s


## Task1.3: 
Write 2 different SQL statements with different conditions to retrieve different rows. 
Explain which parts of the statement are predicate and which parts are projection.

### 0) Prepare print functions

In [11]:
def print_db_file_multiplecolumns_base_one_column_and_equal_value(key, value, columns:list = [], num_rows_to_print:int = 3):
    con = sqlite3.connect(db_file_name)
    cur = con.cursor()
    key_index = csv_columns.index(key)
    key_type = columns_type[key_index]
    
    if columns is None or len(columns) == 0:
        columns = '*'
    else:
        columns = ','.join(columns)
    
    if key_type == 'integer':
        query = f'SELECT {columns} FROM fruits  WHERE {key} = {value}'
    else:
        query = f'SELECT {columns} FROM fruits  WHERE {key} = \'{value}\''
    
    for index, row in enumerate(cur.execute(query)):
        if index >= num_rows_to_print:
            break
        print(row)

    con.close()
    return 

### 1) Predicate Operation : 

> This operation is used to select rows from a table (relation) that specifies a given logic, which is called as a predicate. The predicate is a user defined condition to select rows of user's choice.

> we used the 4 functions above to filter out rows in which different condition were applied

> inside the sql query, the query section "... WHERE {key} = {value}" is what actually filters the wanted rows.

In [12]:
print(f'print only rows in which the color equals red')
print_db_file_multiplecolumns_base_one_column_and_equal_value('color', 'Red')

print only rows in which the color equals red
(3, 'Avocado', 35, 'Red')
(8, 'Avocado', 18, 'Red')
(13, 'Banana', 22, 'Red')


## 2) Project Operation : 

> If the user is interested in selecting the values of a few attributes, rather than selection all attributes of the Table then one should go for PROJECT Operation.  
        
> We used the function above to prject only 2 columns (and also filter some rows)

> inside the sql query, the query section "SELECT {list of columns} FROM ..." is what actually filters the wanted columns

In [13]:
print(f'print only the color and id columns from the rows in which the color equals blue')
print_db_file_multiplecolumns_base_one_column_and_equal_value('color', 'Blue', ['color', 'id'])

print only the color and id columns from the rows in which the color equals blue
('Blue', 0)
('Blue', 7)
('Blue', 9)


## Task 2: CSV and Parquet
1. Write Python program that reads “mydata.csv” file and count number of lines
2. By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapyarrow.parquet”
3. By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”
4. By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”
5. Examine generated Parquet files. Why do you think Dask generated Parquet file differently than PyArrow and Pandas? What might be explanaJon for this?

## Task 2.1:
Write Python program that reads “mydata.csv” file and count number of lines

In [14]:
%%time
# read csv file 
with open(csv_file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    csv_rows = []
    for row in csv_reader:
        if line_count == 0:
            columns_list = row[1::]
        else:
            current_row = row[1::]
            csv_rows.append(current_row)
        line_count+=1

mydata_df = pd.DataFrame(csv_rows, columns = columns_list )

CPU times: user 2.45 s, sys: 147 ms, total: 2.6 s
Wall time: 2.64 s


In [15]:
print('The amount of row in the following csv is when counting with headers: ' + str(line_count))
print('The amount of row in the following csv is when counting without headers: ' + str(len(mydata_df.index)))

The amount of row in the following csv is when counting with headers: 1000001
The amount of row in the following csv is when counting without headers: 1000000


## Task 2.2:
By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapyarrow.parquet”

In [16]:
pyarray_table = pa.Table.from_pandas(mydata_df)
pq.write_table(pyarray_table, parquet_file_name_using_pyarrow)

## Task 2.3
By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”

In [17]:
dask_df = from_pandas(mydata_df, npartitions=4)
dask_df.to_parquet(parquet_file_name_using_dask);

## Task 2.4
By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”

In [18]:
mydata_df.to_parquet(parquet_file_name_using_pandas)

## Task 2.5
Examine generated Parquet files. Why do you think Dask generated Parquet file differently than PyArrow and Pandas? What might be explanation for this?

### Task 2.5 answer

> Unlke parquet files generated from pyarrow and pandas which are saved as a single parquet file, when conerting to parquet using dask we get a folder.

> the parquet file generated from dask is separated into the predefined number of partitions (if we defined npartitions=4, then our dataframe will be split into 4 datasets, which is later saved as 4 parquet files)

> the main eason dask splits the data into 4 sets, is to allow parrallel computing.

## Task 3: Split CSV files
1. Write Python code that calculates size of “mydata.csv” in bytes. Define an integer variable “middle” which is the size of “mydata.csv” divided by 2.
2. Write a Python funcJon first_chunk that count number of rows by reading the byte range of the CSV file, from 0 Jll the “middle”. Write a funcJon last_chunk that count number of rows by reading byte range of CSV file from the “middle”+1 Jll the end of the file.
3. Explain why total number of lines from the first chunk and second chunk is larger than the number of lines calculated in the step (1) of Task 2.
4. Suggest an algorithm to resolve the issue from the step (3) and implement it.
5. Check the algorithm of step (4) with mulJple chunks. Define a chunk size to be 16MB. Write a funcJon that process “mydata.csv “ in chunks and count number of lines for each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so on, until the last chunk, which might be smaller.

## Task 3.1
Write Python code that calculates size of “mydata.csv” in bytes. Define an integer variable “middle” which is the size of “mydata.csv” divided by 2.

In [19]:
memory_usage_size = os.path.getsize(csv_file_name)
middle = memory_usage_size//2
print(f'memory_usage_size of file {memory_usage_size} bytes')
print(f'middle of file {middle} bytes')

memory_usage_size of file 29611308 bytes
middle of file 14805654 bytes


## Task 3.2
Write a Python function first_chunk that count number of rows by reading the byte range of the CSV file, from 0 until the “middle”. Write a funcJon last_chunk that count number of rows by reading byte range of CSV file from the “middle”+1 untill the end of the file.

In [20]:
def first_chunk():
    f2 = open(csv_file_name, "rb")
    f2.seek(0, 0)
    d2 = f2.read(middle).decode(encoding='utf-8')
    amount_of_rows = d2.count('\n')
    
    was_endline = False
    if d2[-1] != '\n':
        amount_of_rows +=1
        was_endline = True
        
    return amount_of_rows, was_endline

def last_chunk():
    f2 = open(csv_file_name, "rb")
    f2.seek(middle+1, 0)
    d2 = f2.read(middle).decode(encoding='utf-8')
    amount_of_rows = d2.count('\n')

    return amount_of_rows

In [21]:
amount_of_rows_first,_ = first_chunk()
amount_of_rows_last = last_chunk()
print(f'amount_of_rows_first {amount_of_rows_first}')
print(f'amount_of_rows_last {amount_of_rows_last}')
print(f'total {amount_of_rows_first + amount_of_rows_last}')

amount_of_rows_first 503738
amount_of_rows_last 496264
total 1000002


## Task 3.3
Explain why total number of lines from the first chunk and second chunk is larger than the number of lines calculated in the step (1) of Task 2.

### Task 3.3 answer

> in this task, the file chunk might end in the middle of a row, and so that row could be counted twice.

> we also note that it matters if you count the number of rows with or without headers of the file. we decided by convention to count with headers

## Task 3.4
Suggest an algorithm to resolve the issue from the step (3) and implement it.

In [22]:
def current_chunk(location, chunk_size):
    f2 = open(csv_file_name, "rb")
    f2.seek(location, 0)
    d2 = f2.read(chunk_size).decode(encoding='utf-8')
    amount_of_rows = d2.count('\n')
    
    was_endline = False
    if d2[-1] != '\n':
        amount_of_rows +=1
        was_endline = True
        
    return amount_of_rows, was_endline

def get_number_of_lines_in_file_v2(file_size, chunk_size:int = 33):
    total_rows = 0
    for location in range(0,file_size,chunk_size):    
        amount_of_rows, was_endline = current_chunk(location, chunk_size)
        total_rows += amount_of_rows
        if was_endline is True:
            total_rows -= 1
        
    return total_rows

## Task 3.5
Check the algorithm of step (4) with multiple chunks. Define a chunk size to be 16MB. Write a function that process “mydata.csv “ in chunks and count number of lines for each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so on, until the last chunk, which might be smaller.

In [23]:
%%time
chunk_size = 16000
result = get_number_of_lines_in_file_v2(memory_usage_size, chunk_size)
print(f'total amount of rows {result}')

total amount of rows 1000001
CPU times: user 60.2 ms, sys: 40.8 ms, total: 101 ms
Wall time: 102 ms
